# ETL Pipeline Preparation
Follow the instructions below to help you create your ETL pipeline.
### 1. Import libraries and load datasets.
- Import Python libraries
- Load `messages.csv` into a dataframe and inspect the first few lines.
- Load `categories.csv` into a dataframe and inspect the first few lines.

In [1]:
# import libraries
import re
import numpy as np
import pandas as pd
from sqlalchemy import create_engine

In [2]:
# load messages dataset
messages = pd.read_csv('./messages.csv')
messages.head()

,id,message,original,genre
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct


In [3]:
# load categories dataset
categories = pd.read_csv('./categories.csv')
categories.head()

,id,categories
0,2,related-1;request-0;offer-0;aid_related-0;medi...
1,7,related-1;request-0;offer-0;aid_related-1;medi...
2,8,related-1;request-0;offer-0;aid_related-0;medi...
3,9,related-1;request-1;offer-0;aid_related-1;medi...
4,12,related-1;request-0;offer-0;aid_related-0;medi...


### 2. Remove duplicates.
- Check how many duplicates are in this dataset.
- Drop the duplicates.
- Confirm duplicates were removed.

In [4]:
# Drop duplicate records
messages.drop_duplicates(inplace=True)
categories.drop_duplicates(inplace=True)

In [5]:
print(messages.shape)
print(categories.shape)

(26180, 4)
(26216, 2)


#### Great, we have made the first pass at removing duplicates, but let's investigate why the number of records is not the same between the two datasets. We can see if there aren't any other duplicates that are hiding in the dataset. We would expect that after removing duplicates that id would be unique for each message. We can do this by seeing if the individual columns are duplicated:

In [6]:
# Show any duplicate ids
messages[messages.id.duplicated(keep=False)]

,id,message,original,genre


In [7]:
#Show any duplicate messages
messages[messages.message.duplicated(keep=False)]

,id,message,original,genre
16555,19448,#NAME?,NaN,news
21749,25213,#NAME?,NaN,news
22527,26062,#NAME?,NaN,news
22737,26309,#NAME?,NaN,news


#### Good news! It appears that the messages data does not have any more actual duplicates. We can see above that nothing was returned when we looked for duplicate ids and the only records returned are logically not duplicates: although the message is the same, it is a generic message with unique id

#### Let's check out the categories data for hidden duplicates

In [8]:
# Show any duplicate ids
categories[categories.id.duplicated(keep=False)]

,id,categories
162,202,related-1;request-1;offer-0;aid_related-1;medi...
163,202,related-1;request-1;offer-0;aid_related-1;medi...
709,862,related-0;request-0;offer-0;aid_related-0;medi...
710,862,related-1;request-0;offer-0;aid_related-0;medi...
1407,1652,related-1;request-1;offer-0;aid_related-1;medi...
...,...,...
24040,27768,related-1;request-0;offer-0;aid_related-0;medi...
24647,28462,related-1;request-1;offer-0;aid_related-1;medi...
24648,28462,related-1;request-0;offer-0;aid_related-1;medi...
25155,29022,related-1;request-1;offer-0;aid_related-1;medi...


#### It appears that there are still some duplicate ids within the categories data that were not identified by the `drop_duplicates` method. We can see that the same id has different labels. For example, the first instance of id 202 has a 1 for aid_related, but a 0 for aid_related in the duplicate instance. Let's investigate these instances further.

In [9]:
messages[messages.id == 28462].iloc[0].message

'To date, a total of 39 people suffering from Ebola or having had a high risk exposure to the virus, have been medically evacuated to Europe.'

In [10]:
first_dup = set(categories[categories.id==28462].categories.iloc[0].split(';'))
second_dup = set(categories[categories.id==28462].categories.iloc[1].split(';'))
first_dup.difference(second_dup)

{'refugees-1', 'request-1'}

#### We can see that one of the duplicates is missing labels for the categories of 'refugees' and 'request'. It seems like message 28462 does pertain to refugees ('evacuated to Europe') and that it is potentially request related. We would probably want to keep the duplicate that has 1 for these categories. Let's look at another example:

In [11]:
messages[messages.id == 1652].iloc[0].message

'please we need water, food and tents, we have 101 chlidren. .. please come rescue'

In [12]:
first_dup = set(categories[categories.id==1652].categories.iloc[0].split(';'))
second_dup = set(categories[categories.id==1652].categories.iloc[1].split(';'))
first_dup.difference(second_dup)

{'search_and_rescue-1'}

#### Again, we can see that one of the duplicates appears to be lacking a relavent label. For id 1652, it looks like one of the duplicates omits a label of 'search_and_rescue', which this message clearly relates to. We would want to keep the duplicate the has a 1 for this label.

#### Overall, what our strategy will be for dealing with these hidden duplicates will be to keep the ones with the most categories labeled 1, based on the assumption that there is a greater risk in false negatives than false positives for classification.

In [13]:
# remove hidden duplicates having same message, but different category counts
count_cats = lambda x: sum([min(int(i),1) for i in re.findall('[0-9]', x)])
categories['cat_counts'] = categories.categories.apply(count_cats)
keep_dict = categories.groupby('id').cat_counts.max().to_dict()
index = categories.id.apply(lambda x: keep_dict[x]) == categories.cat_counts
categories = categories.loc[index].drop('cat_counts', axis=1)

#### Now that we have removed the hidden duplicates, lets check id again...

In [14]:
categories[categories.id.duplicated(keep=False)]

,id,categories
12051,13914,related-2;request-0;offer-0;aid_related-0;medi...
12052,13914,related-1;request-0;offer-0;aid_related-0;medi...
16155,19003,related-1;request-1;offer-0;aid_related-1;medi...
16156,19003,related-1;request-0;offer-0;aid_related-1;medi...


#### It appears there are two remaining duplicate ids. How can this be? Upon further inspection we can see that the reason these ids did not get picked up in our filter is because the labels of each respective id sum to the same amount. So for these ids, the labels are different for the same message. Let's see what these messages are:

In [15]:
messages[messages.id == 13914].iloc[0].message

'Falta tan tan poco.. ¬¨¬¥Simply Red - Holding back the years¬¨¬™. Arena Santiago, de all‚àö¬∞ somos! Night.  http://blip.fm/~n0frf'

In [16]:
first_dup = set(categories[categories.id==13914].categories.iloc[0].split(';'))
second_dup = set(categories[categories.id==13914].categories.iloc[1].split(';'))
first_dup.difference(second_dup)

{'related-2'}

In [17]:
messages[messages.id == 19003].iloc[0].message

"Before we take your general questions, I'd like to make a few opening remarks concerning ISAF support to the Afghan people during these frigid winter months. After the earthquake, huge boulders blocked off the road to large vehicles. Most years, floods wreak havoc in the state, leaving a trail of destruction and killing hundreds of people. We have heard of crocodile sightings."

In [18]:
first_dup = set(categories[categories.id==19003].categories.iloc[0].split(';'))
second_dup = set(categories[categories.id==19003].categories.iloc[1].split(';'))
first_dup.difference(second_dup)

{'cold-0', 'request-1'}

#### It appears the difference between these two messages is not as significant as our previous examples. We will judgmentally drop the second instance of the duplicates.

In [19]:
# remove hidden duplicates having same category count, but different category labels
categories.drop_duplicates(subset='id', inplace=True)

#### Now let's take a look at the shapes of our messages and categories data:

In [20]:
print(messages.shape)
print(categories.shape)

(26180, 4)
(26180, 2)


#### Great, it looks like the number of records are in alignment. We can now move on to expanding the categories and merging the two datasets

### 3. Split `categories` into separate category columns.
- Split the values in the `categories` column on the `;` character so that each value becomes a separate column. You'll find [this method](https://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.Series.str.split.html) very helpful! Make sure to set `expand=True`.
- Use the first row of categories dataframe to create column names for the categories data.
- Rename columns of `categories` with new column names.

In [21]:
# create a dataframe of the 36 individual category columns
categories_expanded = categories.drop('id', axis=1)['categories'].str.split(pat=';', expand=True)

# rename columns
categories_expanded.columns = [col[:-2] for col in categories_expanded.iloc[0,:].tolist()]

categories_expanded

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
1,related-1,request-0,offer-0,aid_related-1,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-1,floods-0,storm-1,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
2,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
3,related-1,request-1,offer-0,aid_related-1,medical_help-0,medical_products-1,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
4,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26243,related-0,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
26244,related-0,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
26245,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
26246,related-1,request-0,offer-0,aid_related-1,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-1,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0


### 4. Convert category values to just numbers 0 or 1.
- Iterate through the category columns in df to keep only the last character of each string (the 1 or 0). For example, `related-0` becomes `0`, `related-1` becomes `1`. Convert the string to a numeric value.
- You can perform [normal string actions on Pandas Series](https://pandas.pydata.org/pandas-docs/stable/text.html#indexing-with-str), like indexing, by including `.str` after the Series. You may need to first convert the Series to be of type string, which you can do with `astype(str)`.

In [22]:
# parse category values to convert them to 0 or 1
categories_clean = pd.DataFrame()
for col in categories_expanded:
    categories_clean[col] = categories_expanded[col].apply(lambda x: min(abs(int(x[-1])),1))
    
categories_clean

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26243,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26244,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26245,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26246,1,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


### 5. Merge datasets.
- Recombine the categories data with id
- Merge the messages and categories datasets using the common id
- Drop uneeded columns
- Assign this combined dataset to `df`, which will be cleaned in the following steps

In [23]:
# recombine cleaned categories data with id #
categories.reset_index(drop=True, inplace=True)
categories_clean.reset_index(drop=True, inplace=True)
categories_clean = pd.concat([categories.id, categories_clean], axis=1)

# merge the messages and categories datasets using the common id and drop unused columns
df = messages.merge(categories_clean, on=['id']).drop(['id','original'], axis=1)
df

,message,genre,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,Weather update - a cold front from Cuba that c...,direct,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Is the Hurricane over or is it not over,direct,1,0,0,1,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,Looking for someone but no name,direct,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"says: west side of Haiti, rest of the country ...",direct,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26175,The training demonstrated how to enhance micro...,news,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26176,A suitable candidate has been selected and OCH...,news,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26177,"Proshika, operating in Cox's Bazar municipalit...",news,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26178,"Some 2,000 women protesting against the conduc...",news,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Lastly, let's check whether any of the columns have no additional information:

In [24]:
df.iloc[:,2:].sum()

related                   20067
request                    4467
offer                       118
aid_related               10843
medical_help               2083
medical_products           1312
search_and_rescue           724
security                    471
military                    860
child_alone                   0
water                      1669
food                       2917
shelter                    2309
clothing                    404
money                       603
missing_people              298
refugees                    875
death                      1192
other_aid                  3442
infrastructure_related     1705
transport                  1199
buildings                  1331
electricity                 532
tools                       159
hospitals                   283
shops                       120
aid_centers                 309
other_infrastructure       1151
weather_related            7286
floods                     2149
storm                      2441
fire    

#### We can see from the output above that our dataset does not have any messages with the label 'child_alone'. Because this label is not providing any additional information, we will drop it from our final cleaned dataset to be used in modeling.

In [25]:
df.drop('child_alone', axis=1, inplace=True)

In [26]:
df.shape

(26180, 37)

### 6. Save the clean dataset into an sqlite database.
You can do this with pandas [`to_sql` method](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_sql.html) combined with the SQLAlchemy library. Remember to import SQLAlchemy's `create_engine` in the first cell of this notebook to use it below.

In [27]:
engine = create_engine('sqlite:///MessagesDatabase.db')
df.to_sql('MessagesDatabase', engine, index=False, if_exists='replace')

### 7. Use this notebook to complete `etl_pipeline.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database based on new datasets specified by the user. Alternatively, you can complete `etl_pipeline.py` in the classroom on the `Project Workspace IDE` coming later.